# Merged PP Dataset

<i>Notes:</i> Last updated 2/23/22. <br>
<br>
<i>To-Do:</i> <br>
- Convert from county to state dataset - still need contraception and domestic violence
- Use indirect standardization to estimate age breakdowns when needed.
- Other data to include: trust in science (scentific funding?), education?, LGBTQ rights?



In [541]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [542]:
DATA_DIR = '../data/'

In [543]:
def check_shape(data):
    # 50 states + DC
    assert data.shape[0] == 51, 'Wrong number of states.'

# States

In [544]:
data = pd.read_csv(DATA_DIR + 'states.csv')
check_shape(data)

In [545]:
data.head(10)

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA
5,Colorado,Colo.,CO
6,Connecticut,Conn.,CT
7,Delaware,Del.,DE
8,District of Columbia,D.C.,DC
9,Florida,Fla.,FL


# Populations
<i>Data source:</i>  <br>
<i>Year:</i> <br>
<i>Notes:</i> To-do - get total population numbers to convert from absolute numbers to percentages.

In [546]:
# pops = pd.read_csv(DATA_DIR + 'population_by_age_by_county.csv')

In [547]:
# pops.head()

# Pregnancy and Abortion
<i>Data source:</i> https://osf.io/td7mk/ <br>
<i>Documentation:</i> https://guttinst.github.io/National-State-Pregnancy-Codebook-1973-2017/ <br>
<i>Year:</i> 2017 <br>
<i>Notes:</i> Rates per 1000, converted to percentage. Dataset also includes miscarriages, births, abortion ratios, earlier years.

In [548]:
pregnancy_abortion = pd.read_csv(DATA_DIR + 'NationalAndStatePregnancy_PublicUse.csv')

# Select lates year only (2017)
pregnancy_abortion = pregnancy_abortion[pregnancy_abortion.year == 2017]

# National data
pregnancy_abortion_usa = pregnancy_abortion[pregnancy_abortion.state == 'US']

# 50 states + DC
pregnancy_abortion = pregnancy_abortion[pregnancy_abortion.state != 'US']

# Check shapes
print(pregnancy_abortion.shape, pregnancy_abortion_usa.shape)
check_shape(pregnancy_abortion)

(51, 103) (1, 103)


In [549]:
# pregnancy_abortion.head(4)

In [550]:
# Select only columsn with pregnancy and abortion rates - keep all age breakdowns for now

# pregnancy_abortion.head(1)
new_keys = ['state']
rename_keys = {}
for key in pregnancy_abortion.keys():
#     print(key)
    if 'pregnancyrate' in key:
        new_keys.append(key)
        rename_keys[key] = 'Pregnancy Rate (%s)' %key[13:]
        pregnancy_abortion[key] /= 10 # Convert to percentage
    elif 'abortionrate' in key:
        new_keys.append(key)
        rename_keys[key] = 'Abortion Rate (%s)' %key[12:]
        pregnancy_abortion[key] /= 10 # Convert to percentage
        
# print(new_keys)

In [551]:
pregnancy_abortion = pregnancy_abortion[new_keys]
pregnancy_abortion = pregnancy_abortion.rename(columns=rename_keys) # Clean up renamed numbers later, if needed

In [552]:
pregnancy_abortion.head(2)

,state,Pregnancy Rate (lt15),Pregnancy Rate (1517),Pregnancy Rate (1819),Pregnancy Rate (1519),Pregnancy Rate (lt20),Pregnancy Rate (2024),Pregnancy Rate (2529),Pregnancy Rate (3034),Pregnancy Rate (3539),...,Abortion Rate (1819),Abortion Rate (1519),Abortion Rate (lt20),Abortion Rate (2024),Abortion Rate (2529),Abortion Rate (3034),Abortion Rate (3539),Abortion Rate (40plus),Pregnancy Rate (total),Abortion Rate (total)
16,AL,0.28,1.63,7.12,3.81,3.87,13.27,14.65,11.32,4.84,...,0.92,0.51,0.53,1.51,1.32,0.82,0.44,0.11,8.33,0.81
33,AK,0.36,1.35,7.02,3.37,3.45,13.50,15.27,13.28,7.11,...,1.22,0.66,0.71,1.68,1.31,1.12,0.58,0.20,9.63,0.97


In [553]:
data = pd.merge(data, pregnancy_abortion, left_on='Code', right_on='state', how='left')
data = data.drop(['state'], axis=1)
check_shape(data)

In [554]:
data.head()

,State,Abbrev,Code,Pregnancy Rate (lt15),Pregnancy Rate (1517),Pregnancy Rate (1819),Pregnancy Rate (1519),Pregnancy Rate (lt20),Pregnancy Rate (2024),Pregnancy Rate (2529),...,Abortion Rate (1819),Abortion Rate (1519),Abortion Rate (lt20),Abortion Rate (2024),Abortion Rate (2529),Abortion Rate (3034),Abortion Rate (3539),Abortion Rate (40plus),Pregnancy Rate (total),Abortion Rate (total)
0,Alabama,Ala.,AL,0.28,1.63,7.12,3.81,3.87,13.27,14.65,...,0.92,0.51,0.53,1.51,1.32,0.82,0.44,0.11,8.33,0.81
1,Alaska,Alaska,AK,0.36,1.35,7.02,3.37,3.45,13.50,15.27,...,1.22,0.66,0.71,1.68,1.31,1.12,0.58,0.20,9.63,0.97
2,Arizona,Ariz.,AZ,0.17,1.35,6.06,3.23,3.26,11.47,13.97,...,1.04,0.54,0.56,1.75,1.52,1.01,0.66,0.24,8.33,0.98
3,Arkansas,Ark.,AR,0.41,1.73,8.40,4.38,4.46,14.26,14.98,...,0.73,0.40,0.41,1.19,1.06,0.63,0.37,0.11,8.51,0.64
4,California,Calif.,CA,0.21,1.29,5.22,2.87,2.91,10.15,12.72,...,1.68,0.96,0.98,2.88,2.53,1.72,1.04,0.37,8.81,1.63


# Contraceptives
<i>Data source:</i> Guttmacher (https://data.guttmacher.org/counties) <br>
<i>Year:</i> 2016 <br>
<i>Notes:</i> Only by-county datasets have age breakdowns. Download each state individually, or use by-state data without age breakdown? "In need of contraceptive services" - what does this mean exactly? (Described [here](https://www.guttmacher.org/report/publicly-supported-FP-services-US-2016).)

In [555]:
contraceptives = pd.read_csv(DATA_DIR + 'guttmacher_need_contraceptive_by_age_2016.csv')
contraceptives.shape

(115, 5)

In [556]:
contraceptives.head()

,County,aged 18-19,younger than 18,younger than 20,Population of women aged 13-44
0,Adair,410,290,710,6900
1,Andrew,140,120,260,3240
2,Atchison,40,30,70,880
3,Audrain,190,150,330,6040
4,Barry,290,210,500,6200


In [557]:
cols = contraceptives.columns
cols_dict = {cols[1]:'Contraception Demand (18-19)', cols[2]:'Contraception Demand (Under 18)', cols[3]:'Contraception Demand (Under 20)', cols[4]:'Female Population (13-44)'}

In [558]:
contraceptives = contraceptives.rename(columns = cols_dict)

In [559]:
contraceptives.head()

,County,Contraception Demand (18-19),Contraception Demand (Under 18),Contraception Demand (Under 20),Female Population (13-44)
0,Adair,410,290,710,6900
1,Andrew,140,120,260,3240
2,Atchison,40,30,70,880
3,Audrain,190,150,330,6040
4,Barry,290,210,500,6200


In [560]:
# data = pd.merge(data, contraceptives, on='County', how='left')
# check_shape(data)

In [561]:
# data[data.State == 'Missouri'].head()

# Domestic Violence
<i>Data source:</i> <br>
<i>Year:</i> 2018 <br>
<i>Notes:</i> MO specific data source, having trouble finding national data.

In [562]:
domestic_violence = pd.read_csv(DATA_DIR + 'domestic_violence_by_county_2018.csv')
domestic_violence.shape

(115, 10)

In [563]:
domestic_violence.head()

,County,Total Domestic,Spouse,Former Spouse,Child in Common,Blood Related,Related by Marriage,Residing Together,Reside in Past,Social Intimate Relationship
0,Adair,320,56,6,31,55,7,114,11,40
1,Andrew,55,22,0,6,6,1,12,3,5
2,Atchison,22,3,1,1,2,2,10,1,2
3,Audrain,498,107,12,9,84,11,31,1,243
4,Barry,394,109,9,7,89,10,143,9,18


In [564]:
cols = domestic_violence.columns
cols_dict = {cols[1]:'Total Domestic Violence (DV)', cols[2]:'DV (Spouse)', cols[3]:'DV (Former Spouse)', cols[4]:'DV (Child in Common)', cols[5]:'DV (Blood Related)', cols[6]:'DV (Related by Marriage)', cols[7]:'DV (Residing Together)', cols[8]:'DV (Resided Together in Past)', cols[9]:'DV (Social Intimate Relationship)'}

In [565]:
domestic_violence = domestic_violence.rename(columns=cols_dict)

In [566]:
domestic_violence.head()

,County,Total Domestic Violence (DV),DV (Spouse),DV (Former Spouse),DV (Child in Common),DV (Blood Related),DV (Related by Marriage),DV (Residing Together),DV (Resided Together in Past),DV (Social Intimate Relationship)
0,Adair,320,56,6,31,55,7,114,11,40
1,Andrew,55,22,0,6,6,1,12,3,5
2,Atchison,22,3,1,1,2,2,10,1,2
3,Audrain,498,107,12,9,84,11,31,1,243
4,Barry,394,109,9,7,89,10,143,9,18


In [567]:
# data = pd.merge(data, domestic_violence, on='County', how='left')
# check_shape(data)

In [568]:
# data[data.State == 'Missouri'].head()

# STD
<i>Data source:</i> <br>
<i>Year:</i> 2019 <br>
<i>Notes:</i> Currently only includes MO.

In [569]:
std = pd.read_csv(DATA_DIR + 'std_age_state.csv')
std.head()

,STD,Age,State,Rate
0,Chlamydia,0-14 years,Alabama (01),0.41
1,Chlamydia,0-14 years,Alaska (02),0.26
2,Chlamydia,0-14 years,Arizona (04),0.20
3,Chlamydia,0-14 years,Arkansas (05),0.32
4,Chlamydia,0-14 years,California (06),0.18


In [570]:
# Clean State strings
states = [si.split('(')[0][:-1] for si in std['State']]
std['State'] = states

In [571]:
# Get unique STD and Age values
ages = std.Age.unique()
stds = std.STD.unique()

In [572]:
# Create new dataframe
std_new = pd.DataFrame()
std_new['State'] = data['State']

In [573]:
# Fill new dataframe with STD x Age rates
for i in range(len(stds)):
    for j in range(len(ages)):
        std_new = pd.merge(std_new, std.loc[(std['STD'] == stds[i]) & (std['Age'] == ages[j])], on='State', how='left')
        std_new = std_new.rename(columns = {'Rate':'%s (%s)' %(stds[i], ages[j])})
        std_new = std_new.drop(['STD', 'Age'], axis=1)
               

In [574]:
check_shape(std_new)
std_new.head()

,State,Chlamydia (0-14 years),Chlamydia (15-19 years),Chlamydia (20-24 years),Chlamydia (25-29 years),Chlamydia (30-34 years),Chlamydia (35-39 years),Chlamydia (40+ years),Gonorrhea (0-14 years),Gonorrhea (15-19 years),...,Gonorrhea (30-34 years),Gonorrhea (35-39 years),Gonorrhea (40+ years),Primary and Secondary Syphilis (0-14 years),Primary and Secondary Syphilis (15-19 years),Primary and Secondary Syphilis (20-24 years),Primary and Secondary Syphilis (25-29 years),Primary and Secondary Syphilis (30-34 years),Primary and Secondary Syphilis (35-39 years),Primary and Secondary Syphilis (40+ years)
0,Alabama,0.41,22.25,23.15,8.54,3.28,1.25,0.20,0.17,8.83,...,2.56,1.43,0.32,0.0,0.06,0.15,0.14,0.12,0.10,0.04
1,Alaska,0.26,22.96,30.30,13.96,6.03,2.86,0.64,0.03,2.23,...,1.62,1.13,0.38,NaN,0.00,0.01,0.02,0.01,0.02,0.01
2,Arizona,0.20,15.59,18.17,8.05,3.71,1.82,0.28,0.04,2.37,...,1.23,0.84,0.19,0.0,0.03,0.10,0.09,0.08,0.08,0.03
3,Arkansas,0.32,18.86,19.12,7.01,2.67,1.00,0.12,0.15,6.69,...,1.73,0.96,0.19,0.0,0.07,0.15,0.13,0.10,0.08,0.02
4,California,0.18,13.58,17.31,8.12,3.73,1.95,0.41,0.04,2.11,...,1.20,0.84,0.24,0.0,0.02,0.08,0.09,0.09,0.09,0.04


In [575]:
data = pd.merge(data, std_new, on='State', how='left')
check_shape(data)

In [576]:
data.head()

,State,Abbrev,Code,Pregnancy Rate (lt15),Pregnancy Rate (1517),Pregnancy Rate (1819),Pregnancy Rate (1519),Pregnancy Rate (lt20),Pregnancy Rate (2024),Pregnancy Rate (2529),...,Gonorrhea (30-34 years),Gonorrhea (35-39 years),Gonorrhea (40+ years),Primary and Secondary Syphilis (0-14 years),Primary and Secondary Syphilis (15-19 years),Primary and Secondary Syphilis (20-24 years),Primary and Secondary Syphilis (25-29 years),Primary and Secondary Syphilis (30-34 years),Primary and Secondary Syphilis (35-39 years),Primary and Secondary Syphilis (40+ years)
0,Alabama,Ala.,AL,0.28,1.63,7.12,3.81,3.87,13.27,14.65,...,2.56,1.43,0.32,0.0,0.06,0.15,0.14,0.12,0.10,0.04
1,Alaska,Alaska,AK,0.36,1.35,7.02,3.37,3.45,13.50,15.27,...,1.62,1.13,0.38,NaN,0.00,0.01,0.02,0.01,0.02,0.01
2,Arizona,Ariz.,AZ,0.17,1.35,6.06,3.23,3.26,11.47,13.97,...,1.23,0.84,0.19,0.0,0.03,0.10,0.09,0.08,0.08,0.03
3,Arkansas,Ark.,AR,0.41,1.73,8.40,4.38,4.46,14.26,14.98,...,1.73,0.96,0.19,0.0,0.07,0.15,0.13,0.10,0.08,0.02
4,California,Calif.,CA,0.21,1.29,5.22,2.87,2.91,10.15,12.72,...,1.20,0.84,0.24,0.0,0.02,0.08,0.09,0.09,0.09,0.04
